<a href="https://colab.research.google.com/github/rubyvanrooyen/astrokat/blob/rvr_observations/notebooks/Astronomy_coordinate_transformation_with_Astropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Astromical Coordinate Systems
https://docs.astropy.org/en/stable/coordinates/index.html

In [9]:
from astropy.coordinates import SkyCoord, AltAz
from astropy.coordinates import Longitude, Latitude, EarthLocation
from astropy.coordinates import Galactic, ICRS
from astropy.time import Time

import astropy.units as u

Do this to get updated IERS B values into astropy

In [10]:
from astropy.utils import iers
from astropy.utils import data
iers_b = iers.IERS_B.open(data.download_file(iers.IERS_B_URL, cache=True))
iers_auto = iers.IERS_Auto.open()

# MeerKAT telescope location

In [11]:
# reference position is given in geodetic coordinates (lat, lon, height)
telescope = EarthLocation.from_geodetic(Longitude('21:26:38.0', u.degree, wrap_angle=180. * u.degree, copy=False),
                                        Latitude('-30:42:39.8', u.degree, copy=False),
                                        height=u.Quantity(1086.6, u.m, copy=False))

In [12]:
print(telescope)

(5109360.13332123, 2006852.58604291, -3238948.12747888) m


# Observation time

In [13]:
# https://docs.astropy.org/en/stable/api/astropy.time.Time.html
# obs_time = Time(obs_time, format='datetime', scale='utc')
#obs_time  = Time('2019-06-12 16:05:18') 
obs_time  = Time('2018-07-23 18:00:00') 
observer = AltAz(location=telescope, obstime=obs_time)

# Equatorial to Horizontal coordinate systems
$(Ra, Dec) \rightarrow (Alt, Az)$

In [14]:
def eq2hor(ra_hms, dec_dms, observer):
    target = SkyCoord(ra=ra_hms, dec=dec_dms, frame='icrs')
    tgt_altaz = target.transform_to(observer)
    return tgt_altaz.alt.deg, tgt_altaz.az.deg

# J1939-6342
name = 'J1939-6342'
ra = '19h39m25.0264s'  # HH:MM:SS.f
dec = '-63d42m45.624s'  # DD:MM:SS.f
print(f'Observer target {name} (RA, Dec) = ({ra}, {dec}) @ {obs_time}')
alt, az = eq2hor(ra, dec, observer)
print(f'Point MKT to (Az, El) = ({az}, {alt})')

Observer target J1939-6342 (RA, Dec) = (19h39m25.0264s, -63d42m45.624s) @ 2018-07-23 18:00:00.000
Point MKT to (Az, El) = (149.43175085428996, 39.36652188083099)


# Horizontal to Equatorial coordinate systems
$(Az, Elev) \rightarrow (Ra, Dec)$

In [15]:
def hor2eq(az_deg, el_deg, location, obstime):
    pointing = AltAz(alt=el_deg*u.deg, az=az_deg*u.deg, location=location, obstime=obstime)
    pnt_radec = pointing.transform_to(ICRS())
    return pnt_radec.ra.hms, pnt_radec.dec.dms

az = 50.26731
elev = 43.70517
print(f'Point MKT to (Az, El) = ({az}, {elev}) @ {obs_time}')
ra, dec = hor2eq(az, elev, telescope, obs_time)
print(f'Observer target {name} (RA, Dec) = ({int(ra.h)}:{int(ra.m)}:{ra.s:.3f}, {int(dec.d)}:{int(dec.m)}:{dec.s:.3f})')

Point MKT to (Az, El) = (50.26731, 43.70517) @ 2018-07-23 18:00:00.000
Observer target J1939-6342 (RA, Dec) = (17:45:46.653, 2:32:54.015)


# Galactic to Equatorial coordinate systems
$(l, b) \rightarrow (Ra, Dec)$

In [16]:
def gal2eq(l, b):
    gal_coord = SkyCoord(l=l, b=b, frame=Galactic)
    print(gal_coord)
    gal_radec = gal_coord.transform_to(ICRS())
    print(gal_radec)
    return gal_radec.ra.hms, gal_radec.dec.dms

name=' T11R00C02'
l =  327.8760*u.deg
b =  0.9877*u.deg
ra, dec = gal2eq(l, b)

print(f'Galactic {name} (l, b) = ({l}, {b})')
print(f'Equatorial {name} (RA, Dec) = ({int(ra.h)}:{int(ra.m)}:{ra.s:.3f}, {int(dec.d)}:{abs(int(dec.m))}:{abs(dec.s):.3f})')

<SkyCoord (Galactic): (l, b) in deg
    (327.876, 0.9877)>
<SkyCoord (ICRS): (ra, dec) in deg
    (237.38390274, -53.03374516)>
Galactic  T11R00C02 (l, b) = (327.876 deg, 0.9877 deg)
Equatorial  T11R00C02 (RA, Dec) = (15:49:32.137, -53:2:1.483)
